In [1]:
import pandas as pd
import numpy as np
import os
import sys
from io import StringIO
import csv

parent = os.path.dirname(os.getcwd())  # workaround to import from utils/
sys.path.append(parent)

from utils.db_utils import df_from_snowflake

In [2]:
herodb_query="""
WITH
    export as (
        SELECT
             keyword
            , sum(search_volume) as sv_last12m
       FROM STAGE.derived_marketing.hero_db_export
       WHERE TRUE
         AND year_month >= dateadd(month, -12, (select max(year_month) from STAGE.derived_marketing.hero_db_export))
         AND blacklisted = 0
         and duplicated is Null
         and duplicated_template is Null
         AND MARKET = '{market}'
       GROUP BY 1
   ),
    master as (
        SELECT
            object_type
            , template
            , keyword
            , object_type_keyword
            , location_id
            , location_name
            , word_form
            , amenity_keyword
            , prefix
        FROM STAGE.derived_marketing.hero_db_keywords_master
        WHERE
            TRUE
            AND MARKET='{market}'
    )
select
    master.*
,   export.sv_last12m
from master
inner join export using (keyword)
where
    TRUE
;
"""


In [3]:
#Load SEM files
def load_sem(market:str):
       df = pd.read_csv('/Users/datnguyen/Downloads/sem_' + market + '.csv', 
                     delimiter='\t', 
                     encoding='utf-16',
                     #nrows=10000,
                     skiprows=2, 
                     decimal='.',
                     thousands=',', 
                     encoding_errors='ignore',
                     on_bad_lines='skip',
                     na_values=' --',
                     usecols=['Search term', 'Impr.', 'Clicks', 'CR', 'ROAS', 'Bookings', 'Revenue'],
                     dtype={'Search term': str, 
                            'Impr.': int, 
                            'Clicks': int, 
                            'CR': str, 
                            'ROAS': str, 
                            'Bookings': float, 
                            'Revenue': str}
                     ).fillna(0)
       print('Search terms of SEM data of ' + market + ': ' + str(len(df)))
       
       #fix type
       df['CR'] = df['CR'].str.replace('%', '').astype(float) / 100
       df['ROAS'] = df['ROAS'].str.replace('%', '').astype(float) / 100
       df['Revenue'] = df['Revenue'].replace({'€': '', ',': ''}, regex=True).astype(float)
       df['Search term'] = df['Search term'].str.lower().str.strip()                              #simple transform before matching
       
       #Groupby Search term because records were on Campaign level
       df_agg = df.groupby(['Search term']).agg({'Impr.': 'sum',
                                                'Clicks': 'sum',
                                                'CR': 'mean',
                                                'ROAS': 'mean',
                                                'Bookings': 'sum',
                                                'Revenue': 'sum'
                                                }).reset_index()
       print('Search terms of SEM after deduplication: ' + str(len(df_agg)))
       return df_agg


#Load HeroDB
def load_herodb(market:str):
       herodb = df_from_snowflake(query=herodb_query.format(market=market))
       herodb['KEYWORD'] = herodb['KEYWORD'].str.lower().str.strip()                               #simple transform before matching
       print('Keywords from HeroDB download is: ' + str(len(herodb)))
       return herodb
       

def merge_df(df_herodb:pd.DataFrame, df_sem:pd.DataFrame):
       new_df = df_herodb.merge(df_sem,
                                how='outer',
                                left_on='KEYWORD',
                                right_on='Search term'
                               )
       
       return new_df


def write_df(df:pd.DataFrame, market:str):
       df.to_csv('/Users/datnguyen/Downloads/herodb_sem_matching_' + market + '.csv',
                 sep= '\t',
                 encoding='utf-8'
                 )
       print(str(len(df)) + ' records have been written')

def make_file(market):
       sem = load_sem(market=market)
       herodb = load_herodb(market=market)
       merged = merge_df(herodb, sem)
       write_df(merged, market)
       return merged



In [4]:
#DE
merged_DE = make_file(market='DE')
print('HeroDB Keywords that matched: ' +  str(len(merged_DE[merged_DE['Search term'].isna()])))


Search terms of SEM data of DE: 10181274
Search terms of SEM after deduplication: 3268297
Keywords from HeroDB download is: 152250


/Users/datnguyen/.pyenv/versions/3.11.5/envs/analytics/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


3325320 records have been written
HeroDB Keywords that matched: 56938


In [13]:
print('HeroDB Keywords that matched: ' +  str(len(merged_DE[(merged_DE['Search term'].isna()==False) & 
                                                            (merged_DE['KEYWORD'].isna()==False)])))

HeroDB Keywords that matched: 95312


In [5]:
#US
merged_US = make_file(market='US')


Search terms of SEM data of US: 17007575
Search terms of SEM after deduplication: 5605498
Keywords from HeroDB download is: 66771


/Users/datnguyen/.pyenv/versions/3.11.5/envs/analytics/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


5635160 records have been written
HeroDB Keywords that matched: 29503


In [14]:
print('HeroDB Keywords that matched: ' +  str(len(merged_US[(merged_US['Search term'].isna()==False) &
                                                            (merged_US['KEYWORD'].isna()==False)])))

HeroDB Keywords that matched: 37268


# Quick look into matching results

In [15]:
#HeroDB keywords without SEM data
len(merged_DE[merged_DE['Search term'].isna()])

56938

In [16]:
len(merged_DE)

3325320

In [15]:
merged_DE

,KEYWORD,OBJECT_TYPE,TEMPLATE,OBJECT_TYPE_KEYWORD,LOCATION_ID,LOCATION_NAME,WORD_FORM,AMENITY_KEYWORD,PREFIX,SV_LAST12M,Search term,Impr.,Clicks,CR,ROAS,Bookings,Revenue
0,pego ferienhaus,All accom KWs DE,[location_name] [object_type],ferienhaus,594b5803e890e,pego,singular,None,None,130.0,pego ferienhaus,5.0,1.0,0.0,0.0,0.0,0.0
1,esch an der alzette hotel,All accom KWs DE,[location_name] [object_type],hotel,5460ae6a33c10,esch an der alzette,singular,None,None,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sesmarias villa,All accom KWs DE,[location_name] [object_type],villa,5460aec4acd41,sesmarias,singular,None,None,70.0,sesmarias villa,3.0,1.0,0.0,0.0,0.0,0.0
3,gjellerodde ferienwohnung,All accom KWs DE,[location_name] [object_type],ferienwohnung,5460aeb07fe48,gjellerodde,singular,None,None,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,geitau pension,All accom KWs DE,[location_name] [object_type],pension,538caddd45429,geitau,singular,None,None,570.0,geitau pension,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,🏡 ferienwohnungen in den bergen,10.0,0.0,0.0,0.0,0.0,0.0
3325316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,🏡 ilha da madeira ilhadamadeira machico,1.0,0.0,0.0,0.0,0.0,0.0
3325317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,📚 booking com,1.0,0.0,0.0,0.0,0.0,0.0
3325318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,🤍 https www booking com hotel de ferienwohnung...,6.0,1.0,0.0,0.0,0.0,0.0


In [18]:
merged_DE[(merged_DE['Search term']=="ferienhaus kroatien mit pool")]

,KEYWORD,OBJECT_TYPE,TEMPLATE,OBJECT_TYPE_KEYWORD,LOCATION_ID,LOCATION_NAME,WORD_FORM,AMENITY_KEYWORD,PREFIX,SV_LAST12M,Search term,Impr.,Clicks,CR,ROAS,Bookings,Revenue
977987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ferienhaus kroatien mit pool,84082.0,19710.0,2.885133,1.008281,129.74,32991.62


In [19]:
herodb_de = load_herodb('DE')

Keywords from HeroDB download is: 151849


In [21]:
herodb_de[herodb_de['KEYWORD']=='ferienhaus kroatien mit pool']

,KEYWORD,OBJECT_TYPE,TEMPLATE,OBJECT_TYPE_KEYWORD,LOCATION_ID,LOCATION_NAME,WORD_FORM,AMENITY_KEYWORD,PREFIX,SV_LAST12M


# Appendix

In [ ]:
#An approach to open file as bytes to fix bugs and save it
file = open('/Users/datnguyen/Downloads/sem_de.csv', 'r', encoding='utf-16')
csvreader = csv.reader(file)



# ignore first 2 rows
next(csvreader)
next(csvreader)

# store other rows
rows = []
for row in csvreader:
        rows.append(row)

file.close()

with open('/Users/datnguyen/Downloads/sem_de_test.csv', 'w', encoding='UTF-8', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    # write multiple rows
    writer.writerows(rows)